# STAC Catalog Setup
CFolkers
Geospatial Services 
2024 02 12

modified from https://github.com/stac-utils/pystac/blob/8079dd3c0cbe8f6f9e48f499ea90f6a5798eaeab/docs/tutorials/how-to-create-stac-catalogs.ipynb
and https://github.com/stac-extensions/pointcloud/blob/main/examples/pdal-to-stac.py

In [1]:
import sys
import logging
import constants
import boto3
from botocore.exceptions import ClientError
import os
from os import path
from pathlib import Path
import pystac 
import pdal
from osgeo import ogr
from osgeo import osr
import json

loading dot env...


In [2]:
# use third party object storage to create an S3 Client
s3_client = boto3.client(
    "s3",
    endpoint_url=constants.AWS_S3_ENDPOINT,
    aws_access_key_id=constants.AWS_ACCESS_KEY_ID,
    aws_secret_access_key=constants.AWS_SECRET_ACCESS_KEY,
)
# for some reason the bucket is adding an extra letter at the end???
bucket = constants.AWS_S3_BUCKET

print(f"{s3_client} {bucket}")

<botocore.client.S3 object at 0x7f17da28a310> rczimv


In [12]:
#list .laz objects in bucket
object_key="STAC_LiDAR/PointClouds/"
laz_objects=[]

response = s3_client.list_objects_v2(Bucket=bucket, Prefix=object_key, StartAfter=object_key)

if 'Contents' in response:
    # Iterate over objects and print their names
    for obj in response['Contents']:
        laz_objects.append(obj['Key'])
        print(obj['Key'])
        print(f"Object Size {obj['Size']}")
else:
    print("No objects found in the bucket.")
    

STAC_LiDAR/PointClouds/bc_092o018_3_2_4_xyes_12_utm10_2018.laz
Object Size 23236093
STAC_LiDAR/PointClouds/bc_092o018_3_4_2_xyes_12_utm10_2018.laz
Object Size 140355729
STAC_LiDAR/PointClouds/bc_092o018_3_4_4_xyes_12_utm10_2018.laz
Object Size 50122462
STAC_LiDAR/PointClouds/bc_092o018_4_1_3_xyes_12_utm10_2018.laz
Object Size 95552259
STAC_LiDAR/PointClouds/bc_092o018_4_1_4_xyes_12_utm10_2018.laz
Object Size 336226672
STAC_LiDAR/PointClouds/bc_092o018_4_3_1_xyes_12_utm10_2018.laz
Object Size 315611463
STAC_LiDAR/PointClouds/bc_092o018_4_3_2_xyes_12_utm10_2018.laz
Object Size 354790466
STAC_LiDAR/PointClouds/bc_092o018_4_3_3_xyes_12_utm10_2018.laz
Object Size 312537985
STAC_LiDAR/PointClouds/bc_092o018_4_3_4_xyes_12_utm10_2018.laz
Object Size 332078905


In [4]:
#One way to access the objects is to Create URL to access .laz file 
url_dict={}
for laz in laz_objects:
    presigned_url=s3_client.generate_presigned_url('get_object',
                                        Params={'Bucket': bucket, 'Key': laz},
                                        ExpiresIn=3600)  # Expiration time in seconds (e.g., 1 hour)
    # print(presigned_url)
    url_dict[laz]=presigned_url

In [5]:
# not working,
# but might be ok once the headers are fixed
#attempt to access laz file form s3 via link

for key in url_dict:

    pipeline = {
        "pipeline": [
            {
                "type": "readers.las",
                "filename":url_dict[key]
            },
            {
                "type": "filters.hexbin"
            },
            {
                "type": "filters.stats"
            },
            {
                "type": "filters.info"
            }
            # Add more processing or filters if needed
        ]
    }
    reader = pdal.Pipeline(json.dumps(pipeline))
    reader.execute()
    boundary = pipeline.metadata['metadata']['filters.hexbin']
    stats = pipeline.metadata['metadata']['filters.stats']
    info = pipeline.metadata['metadata']['filters.info']
    
    break


RuntimeError: readers.las: Invalid VLR offset - exceeds file size.

In [6]:
# not working
#another attempt to read .laz files from s3 link

for key in url_dict.values():
    r = pdal.Reader.copc(key)
    hb = pdal.Filter.hexbin()
    s = pdal.Filter.stats()
    i = pdal.Filter.info()

    pipeline: pdal.Pipeline = r | hb | s | i

    count = pipeline.execute()

    boundary = pipeline.metadata['metadata'][hb.type]
    stats = pipeline.metadata['metadata'][s.type]
    info = pipeline.metadata['metadata'][i.type]
    copc = pipeline.metadata['metadata'][r.type]

RuntimeError: readers.copc: The first VLR in a COPC file is required to have user_id of 'copc' and this file has 'LASF_Projection'

In [7]:
# not working
# another potential way is to use the get_object method 
for laz in laz_objects:
    try:
        response = s3_client.get_object(Bucket=bucket, Key=laz)
        # Access the object data
        object_data = response
        print("Object data:",laz,  object_data)
        r = pdal.Reader.las(response)
        hb = pdal.Filter.hexbin()
        s = pdal.Filter.stats()
        i = pdal.Filter.info()

        pipeline: pdal.Pipeline = r | hb | s | i

        count = pipeline.execute()

        boundary = pipeline.metadata['metadata'][hb.type]
        stats = pipeline.metadata['metadata'][s.type]
        info = pipeline.metadata['metadata'][i.type]
        copc = pipeline.metadata['metadata'][r.type]
    except Exception as e:
        print("Error:", e)
    break

Object data: STAC_LiDAR/PointClouds/bc_092o018_3_2_4_xyes_12_utm10_2018.laz {'ResponseMetadata': {'RequestId': '8e22ee18:1893b9f8f46:2a87bb:2af5', 'HostId': '63b89ba08864215e81252de20042b4b45c2a70ca0f520af1e730ecefa21e44b3', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Mon, 26 Feb 2024 19:28:32 GMT', 'server': 'ViPR/1.0', 'x-amz-request-id': '8e22ee18:1893b9f8f46:2a87bb:2af5', 'x-amz-id-2': '63b89ba08864215e81252de20042b4b45c2a70ca0f520af1e730ecefa21e44b3', 'x-amz-version-id': '1708975587179', 'etag': '"0ac23eae30b35321baa8053e46a54274-3"', 'last-modified': 'Mon, 26 Feb 2024 19:26:27 GMT', 'x-emc-mtime': '1708975587179', 'content-type': 'application/octet-stream', 'content-length': '23236093'}, 'RetryAttempts': 0}, 'LastModified': datetime.datetime(2024, 2, 26, 19, 26, 27, tzinfo=tzutc()), 'ContentLength': 23236093, 'ETag': '"0ac23eae30b35321baa8053e46a54274-3"', 'VersionId': '1708975587179', 'ContentType': 'application/octet-stream', 'Metadata': {}, 'Body': <botocore.response.Strea

In [9]:
def capture_date(pdalinfo):
    import datetime
    year = pdalinfo['creation_year']
    day = pdalinfo['creation_doy']
    date = datetime.datetime(int(year), 1, 1) + datetime.timedelta(int(day) - 1 if int(day) > 1 else int(day))
    return date.isoformat()+'Z'

def convertGeometry(geom, srs):
    from osgeo import ogr
    from osgeo import osr
    in_ref = osr.SpatialReference()
    in_ref.SetFromUserInput(srs)
    out_ref = osr.SpatialReference()
    out_ref.SetFromUserInput('EPSG:4326')

    g = ogr.CreateGeometryFromJson(json.dumps(geom))
    g.AssignSpatialReference(in_ref)
    g.TransformTo(out_ref)
    return json.loads(g.ExportToJson())


def convertBBox(obj):
    output = []
    output.append(float(obj['minx']))
    output.append(float(obj['miny']))
    output.append(float(obj['minz']))
    output.append(float(obj['maxx']))
    output.append(float(obj['maxy']))
    output.append(float(obj['maxz']))
    return output

In [14]:
#download .laz files and read the headers

download_loc=r'/home/cfolkers/STAC_LiDAR/PointClouds/'

for laz in laz_objects:
    output = {}
    
    laz_donwload=f"{download_loc}{laz.split('/')[-1]}"
    
    if not os.path.exists(laz_donwload):   
        s3_client.download_file(bucket, object_key, laz_donwload)
    # #fix WKT 
    # filename = laz_donwload
    # with open(filename, "rb+") as f:
    #     f.seek(6)
    #     f.write(bytes([17, 0, 0, 0]))
    # print (f"Fixed:{filename}")
    
    # #start pdal pipline
    # r = pdal.Reader.las(laz_donwload)
    # hb = pdal.Filter.hexbin()
    # s = pdal.Filter.stats()
    # i = pdal.Filter.info()

    # pipeline: pdal.Pipeline = r | hb | s | i

    # count = pipeline.execute()

    # boundary = pipeline.metadata['metadata'][hb.type]
    # stats = pipeline.metadata['metadata'][s.type]
    # info = pipeline.metadata['metadata'][i.type]
    # copc = pipeline.metadata['metadata'][r.type]
    
    # try:
    #     output['geometry'] = convertGeometry(
    #         boundary['boundary_json'],
    #         copc['comp_spatialreference']
    #     )
    # except KeyError:
    #     output['geometry'] = stats['bbox']['EPSG:4326']['boundary']

    # output['bbox'] = convertBBox(stats['bbox']['EPSG:4326']['bbox'])
    # output['id'] = path.basename(filename)
    # output['type'] = 'Feature'

    # assets = {'data': {'href': filename}}
    # properties = {}

    # properties['pc:schemas'] = info['schema']['dimensions']
    # properties['pc:statistics'] = stats['statistic']
    # properties['title'] = "USGS 3DEP LiDAR"
    # properties['providers'] = [
    #     {
    #         "name": "USGS",
    #         "description": "United States Geological Survey",
    #         "roles": [
    #         "producer",
    #         ],
    #         "url": "https://www.usgs.gov"
    #     }
    # ]
    # properties['pc:type'] = 'lidar' # eopc, lidar, radar, sonar
    # try:
    #     properties['pc:density'] = boundary['avg_pt_per_sq_unit']
    # except KeyError:
    #     properties['pc:density'] = 0
    # properties['pc:count'] = count

    # properties['datetime'] = capture_date(copc)

    # output['properties'] = properties
    # output['assets'] = assets
    # output['stac_extensions'] = ['https://stac-extensions.github.io/pointcloud/v1.0.0/schema.json']
    # output['stac_version'] = '1.0.0'

    # example_dir = Path(__file__).parent
    # out_filename = str(example_dir/'example-autzen.json')

    # self_link = {'rel':'self',"href":'./example-autzen.json'}
    # lic_link = {'rel':'license',"href":'https://github.com/PDAL/data/blob/master/LICENSE'}
    # output['links'] = [self_link, lic_link]
    
    # with open(out_filename, 'w') as autzen_out:
    #     autzen_out.write(json.dumps(output, sort_keys=True, indent=2,
    #                                 separators=(',', ': ')))
    
    break

In [ ]:
?pystac.Catalog

In [4]:
catalog = pystac.Catalog(id="lidar-test", description="Test catalog for the potential use of STAC to access open LiDAR Data")